In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data=pd.read_csv("/content/fake_news.csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
data.shape

(20800, 5)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [5]:
data.isna().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [6]:
data=data.drop(['id'],axis=1)

In [8]:
data=data.fillna('')

In [9]:
data['content']=data['author']+' '+data['title']+' '+data['text']

In [10]:
data=data.drop(['title','author','text'],axis=1)

In [11]:
data.head()

,label,content
0,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,1,Consortiumnews.com Why the Truth Might Get You...
3,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,1,Howard Portnoy Iranian woman jailed for fictio...


In [12]:
data.tail()

,label,content
20795,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."
20799,1,David Swanson What Keeps the F-35 Alive Davi...


In [13]:
data.shape

(20800, 2)

In [14]:
data['content']=data['content'].apply(lambda x:" ".join(x.lower() for x in x.split()))

In [15]:
data['content']=data['content'].str.replace('[^\w\s]','')

In [19]:
from nltk.corpus import stopwords
stop=stopwords.words('english')
data['content']=data['content'].apply(lambda x:" ".join(x for x in x.split() if x not in stop))

In [18]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
from nltk.stem import WordNetLemmatizer
from textblob import Word
data['content']=data['content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['content'].head()

,content
0,darrell lucus house dem aide: didn’t even see ...
1,"daniel j. flynn flynn: hillary clinton, big wo..."
2,consortiumnews.com truth might get fired truth...
3,jessica purkiss 15 civilian killed single u ai...
4,howard portnoy iranian woman jailed fictional ...


In [24]:
from nltk.stem import WordNetLemmatizer
from textblob import Word

data['content'] = data['content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['content'].head()


,content
0,darrell lucus house dem aide: didn’t even see ...
1,"daniel j. flynn flynn: hillary clinton, big wo..."
2,consortiumnews.com truth might get fired truth...
3,jessica purkiss 15 civilian killed single u ai...
4,howard portnoy iranian woman jailed fictional ...


In [22]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [25]:
X=data['content']
Y=data['label']

In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [27]:

X = data[['content']]
Y = data['label']

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
tfidf_vect=TfidfVectorizer(analyzer='word',token_pattern=r'\w{1}',max_features=5000)
tfidf_vect.fit(data['content'])
xtrain_tfidf=tfidf_vect.transform(X_train)

xtest_tfidf=tfidf_vect.transform(X_test)

In [39]:
from sklearn.linear_model  import PassiveAggressiveClassifier
from sklearn import metrics
pclf=PassiveAggressiveClassifier()
pclf.fit(xtrain_tfidf,Y_train)
predictions=pclf.predict(xtest_tfidf)
print(metrics.classification_report(Y_test,predictions))

              precision    recall  f1-score   support

           0       0.72      0.68      0.70      2132
           1       0.68      0.72      0.70      2028

    accuracy                           0.70      4160
   macro avg       0.70      0.70      0.70      4160
weighted avg       0.70      0.70      0.70      4160



In [42]:
print(metrics.confusion_matrix(Y_test,predictions))

[[1445  687]
 [ 562 1466]]


In [48]:
from sklearn.neural_network import MLPClassifier
mlpcf=MLPClassifier(hidden_layer_sizes=(256,64,16),
                    activation='relu',
                    solver='adam')
mlpcf.fit(xtrain_tfidf,Y_train)
predictions=mlpcf.predict(xtest_tfidf)
print(metrics.classification_report(Y_test,predictions))

              precision    recall  f1-score   support

           0       0.77      0.79      0.78      2132
           1       0.77      0.76      0.77      2028

    accuracy                           0.77      4160
   macro avg       0.77      0.77      0.77      4160
weighted avg       0.77      0.77      0.77      4160



/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [50]:
print(metrics.confusion_matrix(Y_test,predictions))

[[1676  456]
 [ 487 1541]]


In [51]:
import pickle
pickle.dump(mlpcf,open("fakenews1.pkl","wb"))